In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.preprocessing import ICA
from mne import concatenate_epochs
from mne.preprocessing import find_eog_events
from mne.preprocessing import corrmap
import pickle as pkl

from Cho2017 import SubjectData

pickle_cache_dir = 'denoise_classify_data'
subject_data_pickle_path = lambda n: os.path.join(pickle_cache_dir, 'data_{}.pkl'.format(n))
subject_ica_pickle_path = lambda n: os.path.join(pickle_cache_dir, 'ica_{}.pkl'.format(n))


plt.rcParams['figure.figsize'] = [12, 8]

# # Make sure ipympl is installed (via pip) for interactive plots in Jupyter 
# %matplotlib widget

In [2]:
ICA_SEED = 97 # Just some random seed
CORRMAP_THRESH = 0.8
n_ica_comps = 64
n_template_ica_comps = 10

In [6]:
exclude_ica_components = {}

for subj_n in subjects:
    print("\n\n========================================")
    print('Subject {:02d}'.format(subj_n))
    print("========================================")
    if os.path.exists(subject_data_pickle_path(subj_n)):
        with open(subject_data_pickle_path(subj_n), 'rb') as f:
            print('Loading', subject_data_pickle_path(subj_n))
            subject = pkl.load(f)
    else:
        subject = SubjectData('../../../Data/mat_data/s{:02d}.mat'.format(subj_n))
        with open(subject_data_pickle_path(subj_n), 'wb') as f:
            print('Reading', subject_datapickle_path(subj_n))
            pkl.dump(subject, f)
    
    im_left_epochs = subject.get_epochs('imagery_left', tmin=-0.5, tmax=2, picks=['eeg'], filter_freqs=(1, None), reject_criteria=None, verbose=False)
    im_right_epochs = subject.get_epochs('imagery_right', tmin=-0.5, tmax=2, picks=['eeg'], filter_freqs=(1, None), reject_criteria=None, verbose=False)
    
    
    
    ############################################################
    # 
    # Get motor imagery ICA components 
    #
    ############################################################
    joint_im_epochs = mne.concatenate_epochs([im_left_epochs.copy(), im_right_epochs.copy()], add_offset=True)
    labels = np.hstack((im_left_epochs.events[:, -1] - 1, im_right_epochs.events[:, -1]))  # Set left events to 0 to have 2 distinct class labels (left: 0 and right: 1)
    
    if os.path.exists(subject_ica_pickle_path(subj_n)):
        with open(subject_ica_pickle_path(subj_n), 'rb') as f:
            print('Loading', subject_ica_pickle_path(subj_n))
            ica_joint = pkl.load(f)
    else:
        ica_joint = mne.preprocessing.ICA(n_components=n_ica_comps, random_state=ICA_SEED, max_iter=1000)
        ica_joint.fit(joint_im_epochs)
        with open(subject_ica_pickle_path(subj_n), 'wb') as f:
            print('Writing', subject_ica_pickle_path(subj_n))
            pkl.dump(ica_joint, f)

    
    # ica_joint.plot_components();
    
    
    ############################################################
    # 
    # Get blinking artifact ICA components 
    #
    ############################################################
    picks = ['FP1', 'FPZ', 'FP2', 'F3', 'F4'] # None / ['eeg'] for first 20 channels or list of channel names for those specified channels

    # Since blinking is performed in 2x 5s trials, it's possible the discontinuity at the trial boundary may be falsely marked as an EOG
    blinking = [subject.raw_noise['blinking'].copy().crop(tmin=0, tmax=5), subject.raw_noise['blinking'].copy().crop(tmin=5, tmax=None)]
    # blinking[0].copy().pick(picks).plot();
    # blinking[1].copy().pick(picks).plot();

    find_eog_ch = 'FP1'
    tmin = -0.25
    tmax = 0.25
    baseline_toffset = 0.01

    trial0_blink_events = find_eog_events(blinking[0], ch_name=find_eog_ch, filter_length=2561)    # Match filter length to length of signal
    trial1_blink_events = find_eog_events(blinking[1], ch_name=find_eog_ch, filter_length=2560)
    
    filter_freqs = (1, None)
    filter_props = dict(picks=['eeg'], fir_design='firwin', skip_by_annotation='edge')
    trial0_blink_epochs = mne.Epochs(blinking[0], trial0_blink_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True).filter(*filter_freqs, **filter_props)
    trial1_blink_epochs = mne.Epochs(blinking[1], trial1_blink_events, tmin=tmin, tmax=tmax, baseline=(tmin, tmin+baseline_toffset), reject=None, preload=True).filter(*filter_freqs, **filter_props)
    # trial0_blink_epochs.plot(picks=picks);
    # trial1_blink_epochs.plot(picks=picks);

    blink_epochs = concatenate_epochs([trial0_blink_epochs, trial1_blink_epochs])
    # blink_epochs.plot(picks=picks);

    ica_blink = mne.preprocessing.ICA(n_components=n_ica_comps, random_state=ICA_SEED, max_iter=1000)
    ica_blink.fit(blink_epochs)
    # ica_blink.plot_components();
    
   
    ############################################################
    # 
    # Get Correlation map across MI/blinking ICA components
    #
    ############################################################
    for i in range(n_template_ica_comps):   # ICA components are sorted by explained variange (R^2), so we only need to take the first few components
        try:
            corrmap([ica_joint], ica_blink.get_components()[:, i], label='test', show=False, plot=False, threshold=CORRMAP_THRESH)
        except Exception as e:
            print(e)
    
    print(ica_joint.labels_['test'])
    ica_joint.plot_components(picks=ica_joint.labels_['test']);
    exclude_ica_components[subj_n] = ica_joint.labels_['test']



Subject 18


NameError: name 'subject_datapickle_path' is not defined

In [4]:
print(exclude_ica_components)

{49: [0, 3], 50: [0, 12], 51: [0], 52: [0, 3]}


In [5]:
import pickle as pkl

with open('blinking-ica_excludes_SEED-{}_THRESH-{}.pkl'.format(ICA_SEED, CORRMAP_THRESH), 'wb') as f:
    pkl.dump(exclude_ica_components, f)

In [6]:
with open('blinking-ica_excludes_SEED-{}_THRESH-{}.pkl'.format(ICA_SEED, 0.9), 'rb') as f:
    _exclude_ica_components = pkl.load(f)

exclude_ica_components = {**_exclude_ica_components, **exclude_ica_components}

with open('blinking-ica_excludes_SEED-{}.pkl'.format(ICA_SEED, CORRMAP_THRESH), 'wb') as f:
    pkl.dump(exclude_ica_components, f)


In [8]:
print(exclude_ica_components)

{1: [0], 2: [0, 2], 3: [0], 4: [1], 5: [0, 32, 2, 5, 6, 9, 43, 11, 14, 15, 16, 17, 18, 23, 57, 29], 6: [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 14, 16, 18, 20, 27, 29, 31, 32, 34, 36], 7: [0], 8: [0], 9: [0, 1], 10: [4], 11: [0], 12: [3], 13: [0, 12], 14: [0], 15: [0], 16: [0], 17: [0], 18: [0], 19: [0], 20: [3], 21: [4], 22: [0], 23: [0], 24: [0], 25: [0, 1, 2, 13, 27], 26: [0], 27: [0], 28: [0], 29: [0], 30: [0], 31: [1], 32: [0], 33: [0, 1, 3, 4, 7, 9, 10, 11, 12, 14, 15, 17, 18, 19, 20, 24, 27, 31, 33, 35, 36, 39, 42, 46, 52, 54], 34: [1], 35: [0, 11], 36: [0], 37: [0, 26], 38: [0], 39: [0], 40: [0], 41: [0], 42: [0], 43: [0, 3, 14], 44: [0], 45: [0], 46: [1], 47: [0, 9, 2], 48: [0], 49: [0, 3], 50: [0, 12], 51: [0], 52: [0, 3]}
